In [0]:
pip install json5

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka_apr2021.ca");

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas")


In [0]:
dbutils.fs.mkdirs("/FileStore/tables/stream/input/transactions")

In [0]:
%fs rm -r /tmp/stream/folder2kafka/*

In [0]:
filePath = "/FileStore/tables/stream/input/transactions/transaction.csv"
read = spark.read.format('csv').options(header='true').options(inferSchema='true').load(filePath)
read.show()
read.printSchema()

In [0]:
from pyspark.sql.types import *
from pyspark.sql.types import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import json5 as json
from pyspark.sql.functions import col

kfk_topic_out = "1agne3o4-transactions-input" #remember to create in cloudkarfka
kfk_brokers = "omnibus-01.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094"
sasl_protocol = "SASL_SSL"
sasl_mech = "SCRAM-SHA-256"
sasl_username = "1agne3o4"
sasl_password = "hlifEhEaV6Gk3Fbu9m_cWzlhTezWUof6"

source_folder = "/FileStore/tables/stream/input/transactions/"
#t_id,t_timestamp,t_user,t_item,t_quantity
#T_ID,T_TS,T_U,T_I,T_Q
print("source_folder:", source_folder)

trans_struct = StructType([ StructField("t_id", StringType(), True), StructField("t_timestamp", TimestampType(), True), StructField("t_user", StringType(), True),StructField("t_item", StringType(), True), StructField("t_quantity", IntegerType(), True) ])
#options(inferSchema='true') - since we are specifying schema(trans_struct)
print("trans_struct:", trans_struct)

trans_df = spark.readStream.options(header='true').csv(source_folder, schema = trans_struct)
print("trans_df(1):", type(trans_df), trans_df)

trans_df = trans_df.selectExpr("to_json(struct(*)) AS value")
print("trans_df(2):", type(trans_df), trans_df)

"""
trans_df.writeStream.format("console").outputMode("append").start().awaitTermination()
"""
#        .option("sasl.jaas.config", 'org.apache.kafka.common.security.plain.PlainLoginModule required username="1agne3o4" password="hlifEhEaV6Gk3Fbu9m_cWzlhTezWUof6"')\

trans_df.writeStream.format("kafka")\
        .option("checkpointLocation", "/tmp/stream/folder2kafka/")\
        .option("kafka.bootstrap.servers", kfk_brokers)\
        .option("kafka.security.protocol", sasl_protocol) \
        .option("security.protocol", sasl_protocol) \
        .option("kafka.sasl.mechanism", sasl_mech) \
        .option("sasl.mechanisms", sasl_mech) \
        .option("topic", kfk_topic_out)\
        .start()\
        .awaitTermination()